# 实现腐蚀样本区域的提取

In [1]:
import cv2
import numpy as np

## 样本提取

In [2]:
rawImage = cv2.imread(r'DSCN9383.JPG')

In [3]:
blurred = cv2.imread(r'test2.jpg')

In [ ]:
#直接画轮廓
rawImage1 = cv2.imread(r'DSCN9358.JPG',0)
equ = cv2.equalizeHist(rawImage1)
contours, hierarchy = cv2.findContours(equ, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE) 

image = cv2.drawContours(rawImage, contours, -1, (0, 0, 255), 3)
cv2.namedWindow('image', 0)
cv2.imshow('image', image)
cv2.waitKey(0)
cv2.destroyAllWindows()

实现思路：
1.高斯模糊
2.图片灰度化
3.Sobel算子
4.图像二值化
5.闭操作
6.膨胀腐蚀
7.中值滤波
8.查找轮廓
9.判断车牌区域

In [ ]:
#GaussianBlur去除部分的干扰
blurred = cv2.GaussianBlur(rawImage, (3, 3), 0)
#img = rawImage - blurred

In [4]:
#图像灰度
gray = cv2.cvtColor(blurred, cv2.COLOR_RGB2GRAY)

In [5]:
#Sobel算子，图像边缘检测，
Sobel_x = cv2.Sobel(gray, cv2.CV_16S, 1, 0)  #（x方向）
Sobel_y = cv2.Sobel(gray, cv2.CV_16S, 0, 1)  #（y方向）

gradX = cv2.convertScaleAbs(Sobel_x)  # 转回uint8
gradY = cv2.convertScaleAbs(Sobel_y)

gradient = cv2.addWeighted(gradX, 0.5, gradY, 0.5, 0)
#gradient = gradX
#gradient = gradY

### 第一次二值化，闭操作

In [ ]:
#图像二值化
ret, binary = cv2.threshold(gradient, 35, 255, cv2.THRESH_OTSU) #OTSU会自动找一个灰度直方图双峰之间的合适阈值
#binary = cv2.adaptiveThreshold(gradient, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 51, 2)

In [ ]:
#闭操作，将目标区域连成一个整体，便于后续轮廓的提取
kernelX = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))
image = cv2.morphologyEx(binary, cv2.MORPH_CLOSE, kernelX)

### 第二次二值化，开操作

In [6]:
#图像二值化, for 二次
ret, binary = cv2.threshold(gradient, 0, 255, cv2.THRESH_BINARY_INV) #OTSU会自动找一个灰度直方图双峰之间的合适阈值
#binary = cv2.adaptiveThreshold(gradient, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 51, 2)

In [7]:
#开操作，断开较窄的狭颈和消除细的突出物,for 二次
kernelX = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
image = cv2.morphologyEx(binary, cv2.MORPH_OPEN, kernelX)

### 后续

In [8]:
#膨胀腐蚀，通过膨胀连接相近的图像区域，通过腐蚀去除孤立细小的色块。（闭操作again，细调）
kernelX = cv2.getStructuringElement(cv2.MORPH_RECT, (10, 1))
kernelY = cv2.getStructuringElement(cv2.MORPH_RECT, (1, 9))

image = cv2.dilate(image, kernelX)
image = cv2.erode(image, kernelX)

image = cv2.erode(image, kernelY)
image = cv2.dilate(image, kernelY)

'''两个方向上同时操作效果不如X和Y方向分别操作
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (20, 19))
image = cv2.dilate(image, kernel)
image = cv2.erode(image, kernel)'''

'两个方向上同时操作效果不如X和Y方向分别操作\nkernel = cv2.getStructuringElement(cv2.MORPH_RECT, (20, 19))\nimage = cv2.dilate(image, kernel)\nimage = cv2.erode(image, kernel)'

In [9]:
#中值滤波，滤去噪声
image = cv2.medianBlur(image, 15)
#image = cv2.GaussianBlur(image, (11,11), 0)

In [ ]:
#查找轮廓，并显示
contours, hierarchy = cv2.findContours(image, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE) 
image = cv2.drawContours(rawImage, contours, -1, (0, 0, 255), 3)

In [10]:
#查找轮廓，并显示
contours, hierarchy = cv2.findContours(image, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE) 
c = sorted(contours, key=cv2.contourArea, reverse=True)[0]

# compute the rotated bounding box of the largest contour
rect = cv2.minAreaRect(c)
box = np.int0(cv2.boxPoints(rect))

# draw a bounding box arounded the detected barcode and display the image
#cv2.drawContours(image, [box], -1, (0, 255, 0), 3) 
image = cv2.drawContours(rawImage,  [box], -1, (0, 0, 255), 3)

## 在轮廓图里找轮廓

In [ ]:
rawImage = cv2.imread(r'test1.jpg')
rawImage1 = cv2.imread(r'DSCN9383.JPG')

In [ ]:
#
gray = cv2.cvtColor(rawImage, cv2.COLOR_RGB2GRAY)
image = gray
contours, hierarchy = cv2.findContours(image, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE) 
c = sorted(contours, key=cv2.contourArea, reverse=True)[2]

# compute the rotated bounding box of the largest contour
rect = cv2.minAreaRect(c)
box = np.int0(cv2.boxPoints(rect))

# draw a bounding box arounded the detected barcode and display the image
#image = cv2.drawContours(rawImage1, [box], -1, (0, 255, 0), 3)
image = cv2.drawContours(rawImage1, contours, -1, (0, 255, 0), 3)

In [ ]:
#裁剪区域
for item in contours:
    rect = cv2.boundingRect(item)
    x = rect[0]
    y = rect[1]
    weight = rect[2]
    height = rect[3]
    if weight > 100 and height > 400:
        # 裁剪区域图片
        chepai = rawImage[y:y + height, x:x + weight]
        cv2.imshow('chepai'+str(x), chepai)
        cv2.waitKey(0)
        cv2.destroyAllWindows()

# 边缘检测先取轮廓

①高斯模糊 - GaussianBlur
②灰度转换 - cvtColor
③计算梯度 – Sobel/Scharr
④非最大信号抑制
⑤高低阈值输出二值图像

In [ ]:
import cv2
import numpy as np

## 高低阈值自设

In [ ]:
def edge(img):
    #高斯模糊,降低噪声
    blurred = cv2.GaussianBlur(img, (3, 3), 0)
#灰度图像
    gray = cv2.cvtColor(blurred, cv2.COLOR_RGB2GRAY)
#图像梯度
    xgrad = cv2.Sobel(gray, cv2.CV_16SC1, 1, 0)
    ygrad = cv2.Sobel(gray, cv2.CV_16SC1, 0, 1)
  #计算边缘
    #50和150参数必须符合1：3或者1：2
    edge_output = cv2.Canny(xgrad, ygrad, 35, 105)
    #图一
    cv2.namedWindow('edge', 0)
    cv2.imshow("edge", edge_output)
    return edge_output
    #图二（彩色）
    #dst = cv.bitwise_and(img, img, mask = edge_output)
    #cv.imshow('cedge', dst)

src = cv2.imread('DSCN9383.JPG')
#图三（原图）
#cv2.namedWindow('def', 0)
#cv2.imshow('def', src)
picsave = edge(src)
cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.imwrite('test1.jpg', picsave)
#cv2.waitKey(0)
#cv2.destroyAllWindows()

## 阈值调节器

In [ ]:
def CannyThreshold(lowThreshold):
    detected_edges = cv2.GaussianBlur(gray, (3,3), 0)
    detected_edges = cv2.Canny(detected_edges, lowThreshold, lowThreshold*ratio, apertureSize = kernel_size)
    dst = cv2.bitwise_and(img, img, mask = detected_edges)  # just add some colours to edges from original image.
    cv2.imshow('canny demo', detected_edges)

lowThreshold = 0
max_lowThreshold = 85
ratio = 3
kernel_size = 3

img = cv2.imread('test2.jpg')
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

cv2.namedWindow('canny demo', 0)
cv2.createTrackbar('Min threshold','canny demo',lowThreshold, max_lowThreshold, CannyThreshold)

CannyThreshold(0)  # initialization
if cv2.waitKey(0):
    cv2.destroyAllWindows()

# 效果显示

In [11]:
cv2.namedWindow('image', 0)
cv2.imshow('image', image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
cv2.imwrite('test2.jpg', image)

In [ ]:
#直方图均衡化
img = cv2.imread('test.jpg', 0)
equ = cv2.equalizeHist(img)
cv2.namedWindow('equ', 0)
cv2.imshow('equ',equ)
cv2.waitKey(0)
cv2.destroyAllWindows()